In [1]:
import pandas as pd
from pandas import DataFrame
import matplotlib
import numpy as np

In [2]:
#展示DF属性
def character(df):
    for each in df.columns:
        print ('字段[{}]特征如下：'.format(each))
        print (df[each].describe())
        print (df[each].ftype)
        print (df[each].drop_duplicates().head(10))
        print ('------------------------------\n')

In [3]:
#获取文件编码
import codecs,os,chardet

def get_charset(filepath):
    with open(filepath, 'rb') as f:
        first_line=f.readline()
        if chardet.detect(first_line)['encoding']=='GB2312':
            return 'gb18030'
        return chardet.detect(first_line)['encoding']

In [10]:
#文件路径
file_dir=r'./transport'
# file_dir+='\\'
#['basic_clean_客户.csv', 'basic_clean_异常.csv', 'basic_clean_缴款.csv', 'basic_clean_运单.csv', 'basic_clean_运输.csv']

In [11]:
#数据合并
import glob

allFiles = glob.glob(file_dir + "/*.csv")

list_ = []

for file_ in allFiles:
    encoding=get_charset(file_)
    df = pd.read_csv(file_,index_col=None, header=0, encoding=encoding,dtype=str)
    with open(file_, encoding=encoding) as f:
        print (df.shape[0]-len(f.readlines())+1, end=' ')
    
    list_.append(df)

frame = pd.concat(list_, ignore_index = True)

0 

In [12]:
frame.to_pickle('yunshu')

In [15]:
frame=pd.read_csv('out_khy_waybill_info-01.csv',dtype=str, encoding='gb18030')
frame.to_pickle('waybills')
frame=pd.read_csv('out_khy_payment_info-01.csv',dtype=str, encoding='gb18030')
frame.to_pickle('jiaokuan')
frame=pd.read_csv('out_khy_transport_info-01.csv',dtype=str, encoding='gb18030')
frame.to_pickle('yunshu')

In [16]:
# 诚信运单状态替换及地址补充
waybill = pd.read_pickle('./waybills')
jiaokuan = pd.read_pickle('./jiaokuan')
yunshu = pd.read_pickle('./yunshu')

#诚信网点信息
chenxin_spot={'石家庄':['河北','石家庄'],
    '临沂':['山东','临沂'],
    '阜阳':['安徽','阜阳'],
    '太和':['安徽','太和'],
    '界首':['安徽','界首'],
    '长治':['山西','长治'],
    '晋城':['山西','晋城'],
    '晋城北':['山西','晋城'],
    '高平':['山西','高平'],
    '阳城':['山西','阳城'],
    '运城':['山西','运城'],
    '垣曲':['山西','垣曲'],
    '新郑':['河南','新郑'],
    '中牟':['河南','中牟'],
    '荥阳':['河南','荥阳'],
    '荥阳南':['河南','荥阳'],
    '上街':['河南','上街'],
    '登封':['河南','登封'],
    '巩义':['河南','巩义'],
    '回郭镇':['河南','巩义'],
    '新密':['河南','新密'],
    '开封':['河南','开封'],
    '兰考':['河南','兰考'],
    '固阳镇':['河南','开封'],
    '尉氏':['河南','尉氏'],
    '通许':['河南','通许'],
    '杞县':['河南','杞县'],
    '洛阳':['河南','洛阳'],
    '关林':['河南','关林'],
    '宜阳':['河南','宜阳'],
    '新安':['河南','新安'],
    '伊川':['河南','伊川'],
    '孟津':['河南','孟津'],
    '嵩县':['河南','嵩县'],
    '偃师':['河南','偃师'],
    '洛宁':['河南','洛宁'],
    '平顶山':['河南','平顶山'],
    '平顶山西':['河南','平顶山'],
    '宝丰':['河南','宝丰'],
    '鲁山':['河南','鲁山'],
    '郏县':['河南','郏县'],
    '汝州':['河南','汝州'],
    '舞钢':['河南','舞钢'],
    '叶县':['河南','叶县'],
    '安阳':['河南','安阳'],
    '林州':['河南','林州'],
    '内黄':['河南','内黄'],
    '汤阴':['河南','汤阴'],
    '滑县':['河南','滑县'],
    '鹤壁':['河南','鹤壁'],
    '淇县':['河南','淇县'],
    '浚县':['河南','浚县'],
    '新乡':['河南','新乡'],
    '新乡北':['河南','新乡'],
    '小冀镇':['河南','郑州'],
    '延津':['河南','延津'],
    '封丘':['河南','封丘'],
    '卫辉':['河南','卫辉'],
    '获嘉':['河南','获嘉'],
    '原阳':['河南','原阳'],
    '长垣':['河南','长垣'],
    '辉县':['河南','辉县'],
    '焦作':['河南','焦作'],
    '修武':['河南','修武'],
    '武陟':['河南','武陟'],
    '詹店镇':['河南','焦作市'],
    '温县':['河南','温县'],
    '温县南':['河南','温县'],
    '沁阳':['河南','沁阳'],
    '博爱':['河南','博爱'],
    '孟州':['河南','孟州'],
    '濮阳':['河南','濮阳'],
    '濮阳东':['河南','濮阳'],
    '濮阳县':['河南','濮阳县'],
    '清丰':['河南','清丰'],
    '南乐':['河南','南乐'],
    '许昌':['河南','许昌'],
    '鄢陵':['河南','鄢陵'],
    '长葛':['河南','长葛'],
    '后河镇':['河南','许昌'],
    '禹州':['河南','禹州'],
    '花石镇':['河南','许昌'],
    '襄县':['河南','襄县'],
    '漯河':['河南','漯河'],
    '舞阳':['河南','舞阳'],
    '临颍':['河南','临颍'],
    '三门峡':['河南','三门峡'],
    '灵宝':['河南','灵宝'],
    '陕县':['河南','陕县'],
    '渑池':['河南','渑池'],
    '义马':['河南','义马'],
    '卢氏':['河南','卢氏'],
    '商丘西':['河南','商丘'],
    '商丘东':['河南','商丘'],
    '宁陵':['河南','宁陵'],
    '虞城':['河南','虞城'],
    '永城':['河南','永城'],
    '柘城':['河南','柘城'],
    '睢县':['河南','睢县'],
    '民权':['河南','民权'],
    '周口':['河南','周口'],
    '周口北':['河南','周口'],
    '项城':['河南','项城'],
    '沈丘':['河南','沈丘'],
    '淮阳':['河南','淮阳'],
    '郸城':['河南','郸城'],
    '太康':['河南','太康'],
    '扶沟':['河南','扶沟'],
    '西华':['河南','西华'],
    '鹿邑':['河南','鹿邑'],
    '商水':['河南','商水'],
    '驻马店':['河南','驻马店'],
    '驻马店东':['河南','驻马店'],
    '确山':['河南','确山'],
    '新蔡':['河南','新蔡'],
    '正阳':['河南','正阳'],
    '汝南':['河南','汝南'],
    '平舆':['河南','平舆'],
    '上蔡':['河南','上蔡'],
    '遂平':['河南','遂平'],
    '西平':['河南','西平'],
    '南阳':['河南','南阳'],
    '南阳南':['河南','南阳'],
    '社旗':['河南','社旗'],
    '方城':['河南','方城'],
    '邓州':['河南','邓州'],
    '唐河':['河南','唐河'],
    '新野':['河南','新野'],
    '信阳':['河南','信阳'],
    '罗山':['河南','罗山'],
    '淮滨':['河南','淮滨'],
    '商城':['河南','商城'],
    '固始':['河南','固始'],
    '息县':['河南','息县'],
    '潢川':['河南','潢川'],
    '光山':['河南','光山'],
    '济源':['河南','济源'],
    '总部':['河南','郑州'],
    '郑州':['河南','郑州'],
    '南区运营中心':['河南','郑州'],
    '张庄一部':['河南','郑州'],
    '张庄二部':['河南','郑州'],
    '张庄三部':['河南','郑州'],
    '张庄四部':['河南','郑州'],
    '张庄五部':['河南','郑州'],
    '华丰灯饰分部':['河南','郑州'],
    '家电分部':['河南','郑州'],
    '十里铺分部':['河南','郑州'],
    '吴庄分部':['河南','郑州'],
    '香江分部':['河南','郑州'],
    '九龙朝伟':['河南','郑州'],
    '九龙慧丽':['河南','郑州'],
    '白沙分部':['河南','郑州'],
    '浩瀚建材分部':['河南','郑州'],
    '康庄分部':['河南','郑州'],
    '郑庵分部':['河南','郑州'],
    '郑开分部':['河南','郑州'],
    '中铁二部':['河南','郑州'],
    '鑫中原分部':['河南','郑州'],
    '热电分部':['河南','郑州'],
    '韩庄分部':['河南','郑州'],
    '华丰金属分部':['河南','郑州'],
    '华丰金属二部':['河南','郑州'],
    '华丰金属三部':['河南','郑州'],
    '华丰金属五部':['河南','郑州'],
    '八里湾分部':['河南','郑州'],
    '魏庄分部':['河南','郑州'],
    '下板峪分部':['河南','郑州'],
    '木工市场分部':['河南','郑州'],
    '郑油磨分部':['河南','郑州'],
    '汽博园分部':['河南','郑州'],
    '亚丰五金分部':['河南','郑州'],
    '樱桃沟分部':['河南','郑州'],
    '第一城分部':['河南','郑州'],
    '西环分部':['河南','郑州'],
    '百荣分部':['河南','郑州'],
    '百荣二部':['河南','郑州'],
    '三环一部':['河南','郑州'],
    '三环二部':['河南','郑州'],
    '鑫兴分部':['河南','郑州'],
    '南曹分部':['河南','郑州'],
    '南曹二部':['河南','郑州'],
    '华南城一部':['河南','郑州'],
    '华南城二部':['河南','郑州'],
    '华南城三部':['河南','郑州'],
    '华南城四部':['河南','郑州'],
    '华商汇分部':['河南','郑州'],
    '华商汇一部':['河南','郑州'],
    '华商汇二部':['河南','郑州'],
    '华商汇三部':['河南','郑州'],
    '创业园分部':['河南','郑州'],
    '曲梁分部':['河南','郑州'],
    '大营分部':['河南','郑州'],
    '大营分部':['河南','郑州'],
    '郑邦分部':['河南','郑州'],
    '黄金大道分部':['河南','郑州'],
    '黄金大道二部':['河南','郑州'],
    '薛店机场分部':['河南','郑州'],
    '新澳辉分部':['河南','郑州'],
    '八堡一部':['河南','郑州'],
    '万客隆分部':['河南','郑州'],
    '临沂中转':['河南','郑州'],
    '正定中转':['河南','郑州'],
    '南浔中转':['河南','郑州'],          
    }

def chenxin_trans_status(fazhan,status):
    status_pair = {'已提货': '已签收', '已收款': '已签收', '已送货': '已到站',
               '原返': '已开单', '款已到': '已提货', '出库→': '已到货', '发车': '已发运'}
    for each in status_pair.keys():
        if each in status:
            return status_pair[each]
    if '库' in status:
        return '已到站'
#     for each in chenxin_spot.keys():
#         if each in status:
#             return '已开单'
#     if '郑州' in fazhan:
#         return '已开单'
    return '已开单'      
    # 已提货——已签收
    # 已收款——已签收
    # 已送货——已到站
    # 原返——已开单
    # 款已到——已提货
    # 出库→三环宫宇库——已到货
    # 【网点】库——已到货
    # 发车——已发运
    # 【网点】——已开单

#映射运单状态
waybill['运单状态']=waybill[['运单状态','发站']].apply(lambda x:chenxin_trans_status(x[1],x[0]),axis=1)

#映射网点地址
waybill['货号']=waybill['运单号']
waybill['物流公司']='河南诚信'
waybill['发省']=waybill['发站'].map(lambda x:chenxin_spot[x][0] if x in chenxin_spot.keys() else '')
waybill['发市']=waybill['发站'].map(lambda x:chenxin_spot[x][1] if x in chenxin_spot.keys() else '')
waybill['收省']=waybill['到站'].map(lambda x:chenxin_spot[x][0] if x in chenxin_spot.keys() else '')
waybill['收市']=waybill['到站'].map(lambda x:chenxin_spot[x][1] if x in chenxin_spot.keys() else '')
waybill.to_pickle('waybills')


#缴款补齐
jiaokuan['缴款编号']=jiaokuan['运单编号']
jiaokuan['运单数']=1
jiaokuan.to_pickle('jiaokuan')

#运输补齐
yunshu['发车批次']=yunshu['运单编号']
yunshu.to_pickle('yunshu')

In [17]:
waybill.to_csv('waybill.csv',index=False)
jiaokuan.to_csv('jiaokuan.csv',index=False)
yunshu.to_csv('yunshu.csv',index=False)

In [18]:
character(waybill)

字段[运单号]特征如下：
count       236483
unique      236481
top       70267120
freq             2
Name: 运单号, dtype: object
object:dense
0    20754513
1    20898233
2    20960057
3    20967437
4    75076326
5    75076327
6    20967440
7    20967441
8    20967442
9    20967443
Name: 运单号, dtype: object
------------------------------

字段[运单状态]特征如下：
count     236493
unique         6
top          已签收
freq      222984
Name: 运单状态, dtype: object
object:dense
0       已签收
26      已发运
32      已到货
122     已到站
282     已开单
1653    已提货
Name: 运单状态, dtype: object
------------------------------

字段[运单创建时间]特征如下：
count              236493
unique              26369
top       2019/5/16 17:29
freq                   43
Name: 运单创建时间, dtype: object
object:dense
0     2019/4/5 12:20
1     2019/4/23 8:29
2    2019/4/30 13:39
3      2019/5/1 1:47
4      2019/5/1 2:45
5      2019/5/1 2:46
6      2019/5/1 7:19
7      2019/5/1 7:34
8      2019/5/1 7:38
9      2019/5/1 7:40
Name: 运单创建时间, dtype: object
--------------------------